In [1]:
# core
import numpy as np
import pandas as pd
# from scipy.interpolate import CubicSpline

# astronomy
import astropy
from astropy.units import deg, au, km, meter, day, minute, second
# from astropy.coordinates import SkyCoord, ICRS, GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic, EarthLocation
# import skyfield
# from skyfield.api import Loader
# skyfield_load = Loader('../data/skyfield')
# from skyfield.positionlib import ICRF, Barycentric
# from skyfield.toposlib import Topos

# utility
from datetime import datetime
# import os
# import matplotlib.pyplot as plt

# MSE imports
# import astro_utils
from utils import range_inc
from astro_utils import jd_to_mjd, mjd_to_jd, datetime_to_mjd
from ra_dec import radec2dir, dir2radec, site2geoloc, qv2dir, direction_diff, radec_diff, skyfield_observe
from ra_dec import calc_topos, astrometric_dir
from horizons_files import load_pos_jpl, load_ast_jpl, load_obs_jpl, load_obs_ast_jpl
from horizons_files import obs_add_interp_qv, obs_ast_add_interp_qv, obs_add_calc_dir, obs_add_radec, obs_direction_diff
# import asteroid_integrate
# from asteroid_data import make_data_one_file, get_earth_pos, load_data
from asteroid_data import load_data
from asteroid_dataframe import load_ast_data, spline_ast_data, spline_ast_obs, compare_df_vec, compare_df_obs

# Types
from typing import Optional

### Position of Earth according to JPL

In [2]:
# Data directories
dir_name_hourly = '../data/jpl/testing/hourly'
dir_name_daily = '../data/jpl/testing/daily'

# Build DataFrame for earth and mars position at 3 hour frequency
# df_earth = load_pos_jpl(body_name='earth', dir_name=dir_name_hourly)

# Earth at daily frequency
df_earth_jpl = load_pos_jpl(body_name='earth', dir_name=dir_name_daily)

In [3]:
df_earth_jpl

,mjd,JulianDate,time_key,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,55197.0,2455197.5,1324728,-0.179765,0.970347,-0.000017,-0.017202,-0.003148,8.961125e-07,0.005700,0.986858,0.000038
1,55198.0,2455198.5,1324752,-0.196939,0.967049,-0.000017,-0.017145,-0.003447,9.036109e-07,0.005700,0.986899,0.000044
2,55199.0,2455199.5,1324776,-0.214053,0.963453,-0.000016,-0.017083,-0.003745,8.653246e-07,0.005700,0.986945,0.000049
3,55200.0,2455200.5,1324800,-0.231103,0.959559,-0.000015,-0.017017,-0.004042,7.855759e-07,0.005700,0.986997,0.000054
4,55201.0,2455201.5,1324824,-0.248085,0.955369,-0.000014,-0.016945,-0.004339,6.725245e-07,0.005701,0.987054,0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...
3648,58845.0,2458845.5,1412280,-0.100787,0.986067,-0.000022,-0.017416,-0.001767,9.809367e-07,0.005725,0.991204,0.000013
3649,58846.0,2458846.5,1412304,-0.118187,0.984147,-0.000021,-0.017382,-0.002073,9.327289e-07,0.005725,0.991218,0.000015
3650,58847.0,2458847.5,1412328,-0.135550,0.981922,-0.000020,-0.017343,-0.002377,8.590659e-07,0.005725,0.991234,0.000017
3651,58848.0,2458848.5,1412352,-0.152871,0.979393,-0.000019,-0.017298,-0.002681,7.650397e-07,0.005725,0.991251,0.000019


### Positions of First 16 Asteroids from JPL

In [4]:
# Load the asteroid position and velocity from JPL
df_ast_jpl = load_ast_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name_daily)

In [5]:
df_ast_jpl

,asteroid_num,mjd,JulianDate,time_key,X,Y,Z,VX,VY,VZ,LT,RG,RR
0,1,55197.0,2455197.5,1324728,-1.660333,-2.123236,0.238962,0.007615,-0.007150,-0.001627,0.015628,2.705909,0.000794
1,1,55198.0,2455198.5,1324752,-1.652706,-2.130370,0.237334,0.007640,-0.007118,-0.001630,0.015633,2.706703,0.000794
2,1,55199.0,2455199.5,1324776,-1.645053,-2.137472,0.235702,0.007665,-0.007086,-0.001634,0.015637,2.707497,0.000795
3,1,55200.0,2455200.5,1324800,-1.637376,-2.144542,0.234066,0.007689,-0.007054,-0.001637,0.015642,2.708293,0.000796
4,1,55201.0,2455201.5,1324824,-1.629675,-2.151580,0.232427,0.007713,-0.007022,-0.001641,0.015646,2.709088,0.000796
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,16,58845.0,2458845.5,1412280,2.517677,-0.513079,-0.043698,0.001590,0.011210,-0.000568,0.014842,2.569797,-0.000671
3649,16,58846.0,2458846.5,1412304,2.519245,-0.501865,-0.044266,0.001546,0.011219,-0.000568,0.014838,2.569129,-0.000665
3650,16,58847.0,2458847.5,1412328,2.520770,-0.490641,-0.044834,0.001503,0.011228,-0.000567,0.014834,2.568466,-0.000660
3651,16,58848.0,2458848.5,1412352,2.522251,-0.479409,-0.045400,0.001459,0.011236,-0.000566,0.014830,2.567809,-0.000655


### Observations of First 16 Asteroids from JPL

In [6]:
# Load the asteroid observations from JPL
df_obs_jpl = load_obs_ast_jpl(ast_num0=1, ast_num1=16, observer_name='geocenter', dir_name=dir_name_daily)

In [7]:
df_obs_jpl

,asteroid_num,mjd,JulianDate,time_key,RA_jpl,DEC_jpl,ux_jpl,uy_jpl,uz_jpl,RA_apparent,DEC_apparent,delta,delta_dot,light_time
0,1,55197.0,2455197.5,1324728,243.215442,-17.105913,-0.430702,-0.899812,0.069523,243.358581,-17.131844,3.437877,-12.468091,28.591952
1,1,55198.0,2455198.5,1324752,243.625145,-17.196033,-0.424384,-0.902835,0.069195,243.768548,-17.221645,3.430618,-12.668422,28.531584
2,1,55199.0,2455199.5,1324776,244.034084,-17.284935,-0.418063,-0.905804,0.068867,244.177730,-17.310227,3.423244,-12.868277,28.470254
3,1,55200.0,2455200.5,1324800,244.442231,-17.372621,-0.411741,-0.908720,0.068539,244.586099,-17.397589,3.415755,-13.067723,28.407966
4,1,55201.0,2455201.5,1324824,244.849560,-17.459094,-0.405417,-0.911583,0.068211,244.993632,-17.483728,3.408150,-13.266749,28.344720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,16,58845.0,2458845.5,1412280,332.583400,-12.175911,0.867713,-0.496854,-0.014471,332.842722,-12.079671,3.017646,17.406609,25.096998
3649,16,58846.0,2458846.5,1412304,332.963279,-12.044021,0.871109,-0.490873,-0.014611,333.222363,-11.947459,3.027660,17.268707,25.180280
3650,16,58847.0,2458847.5,1412328,333.344527,-11.910963,0.874478,-0.484841,-0.014750,333.603364,-11.814086,3.037593,17.128990,25.262894
3651,16,58848.0,2458848.5,1412352,333.727108,-11.776749,0.877821,-0.478758,-0.014889,333.985688,-11.679564,3.047446,16.987592,25.344833


### Position of Asteroids & Earth from MSE Integration

In [8]:
# alias inputs
n0 = 1
n1 = 16
mjd0 = datetime_to_mjd(datetime(2010,1,1))
mjd1 = datetime_to_mjd(datetime(2020,1,2))

In [9]:
# Load first block of asteroid data
df_ast_mse, df_earth_mse, df_sun_mse = load_ast_data(n0=n0, n1=n1, mjd0=mjd0, mjd1=mjd1)

In [10]:
# DataFrame of asteroid snapshots
ast_elt = load_data()
# ast_elt

### Check Position of Earth vs. JPL

In [11]:
df_earth_mse

,mjd,time_key,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f
0,55197.0,1324728,-0.179770,0.970346,-0.000018,-0.017202,-0.003150,8.432308e-07,0.999049,0.015785,0.000050,2.230747,-0.423944,-0.056025
1,55198.0,1324752,-0.196943,0.967047,-0.000017,-0.017144,-0.003449,9.008784e-07,0.999023,0.015754,0.000052,2.205479,-0.405460,-0.031460
2,55199.0,1324776,-0.214057,0.963449,-0.000016,-0.017082,-0.003747,9.135747e-07,0.999051,0.015779,0.000053,2.202614,-0.409563,-0.006712
3,55200.0,1324800,-0.231106,0.959554,-0.000015,-0.017015,-0.004044,8.801033e-07,0.999132,0.015860,0.000051,2.220296,-0.433715,0.017540
4,55201.0,1324824,-0.248085,0.955362,-0.000015,-0.016943,-0.004339,8.042460e-07,0.999260,0.015988,0.000046,2.260180,-0.478972,0.040696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,58845.0,1412280,-0.100788,0.986066,-0.000022,-0.017416,-0.001767,9.773340e-07,1.000857,0.017568,0.000062,2.475880,-0.676480,-0.129783
3649,58846.0,1412304,-0.118188,0.984146,-0.000021,-0.017382,-0.002073,9.268695e-07,1.000770,0.017493,0.000059,2.520142,-0.715584,-0.117150
3650,58847.0,1412328,-0.135551,0.981921,-0.000020,-0.017343,-0.002377,8.513399e-07,1.000654,0.017387,0.000056,2.586981,-0.777772,-0.104008
3651,58848.0,1412352,-0.152872,0.979391,-0.000019,-0.017298,-0.002681,7.558548e-07,1.000516,0.017257,0.000052,2.678579,-0.865408,-0.090176


In [12]:
# Run comparison on earth
compare_df_vec(df_mse=df_earth_mse, df_jpl=df_earth_jpl, name='earth')

Mean absolute error for df_earth_mse vs. df_earth_jpl:
mjd: 0.00e+00 days
  q: 3.92e-06 AU     (max 1.13e-05)
  v: 8.57e-07 AU/day (rel 4.98e-05)


### Check Position of Asteroids vs. JPL

In [13]:
df_ast_mse

,asteroid_num,mjd,time_key,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f
0,1,55197.0,1324728,-1.660334,-2.123236,0.238962,0.007615,-0.007150,-0.001627,2.765732,0.079223,0.184765,1.403138,1.268764,1.371390
1,1,55198.0,1324752,-1.652706,-2.130370,0.237334,0.007640,-0.007118,-0.001630,2.765729,0.079223,0.184765,1.403138,1.268752,1.375296
2,1,55199.0,1324776,-1.645054,-2.137472,0.235702,0.007665,-0.007086,-0.001634,2.765726,0.079222,0.184765,1.403138,1.268740,1.379200
3,1,55200.0,1324800,-1.637377,-2.144542,0.234067,0.007689,-0.007054,-0.001637,2.765723,0.079222,0.184765,1.403137,1.268728,1.383101
4,1,55201.0,1324824,-1.629676,-2.151580,0.232428,0.007713,-0.007022,-0.001641,2.765720,0.079221,0.184765,1.403137,1.268717,1.387000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58443,16,58845.0,1412280,2.517677,-0.513080,-0.043698,0.001590,0.011210,-0.000568,2.922647,0.133480,0.054046,2.618716,-2.286344,-0.535517
58444,16,58846.0,1412304,2.519246,-0.501866,-0.044266,0.001546,0.011219,-0.000568,2.922646,0.133480,0.054046,2.618716,-2.286339,-0.531126
58445,16,58847.0,1412328,2.520770,-0.490642,-0.044834,0.001503,0.011228,-0.000567,2.922645,0.133480,0.054046,2.618716,-2.286333,-0.526732
58446,16,58848.0,1412352,2.522251,-0.479410,-0.045400,0.001459,0.011236,-0.000566,2.922644,0.133480,0.054046,2.618716,-2.286328,-0.522336


In [14]:
# Filter MSE asteroids down to just the first 16 to match JPL data
mask = df_ast_mse.asteroid_num <= 16
df_ast_mse_16 = df_ast_mse[mask]

In [15]:
# Run comparison on asteroids
compare_df_vec(df_mse=df_ast_mse_16, df_jpl=df_ast_jpl, name='asteroids')

Mean absolute error for df_asteroids_mse vs. df_asteroids_jpl:
mjd: 0.00e+00 days
  q: 7.90e-07 AU     (max 3.52e-06)
  v: 2.55e-09 AU/day (rel 2.38e-07)


### Review Solar DataFrame

In [16]:
df_sun_mse

,mjd,time_key,qx,qy,qz,vx,vy,vz
0,55197.0,1324728,-0.003747,0.002926,0.000004,-0.000003,-0.000006,6.981105e-08
1,55198.0,1324752,-0.003750,0.002921,0.000005,-0.000003,-0.000006,6.965225e-08
2,55199.0,1324776,-0.003753,0.002915,0.000005,-0.000003,-0.000006,6.949627e-08
3,55200.0,1324800,-0.003756,0.002910,0.000005,-0.000003,-0.000006,6.934233e-08
4,55201.0,1324824,-0.003759,0.002904,0.000005,-0.000003,-0.000006,6.918960e-08
...,...,...,...,...,...,...,...,...
3648,58845.0,1412280,-0.003765,0.007447,0.000022,-0.000008,-0.000002,2.306296e-07
3649,58846.0,1412304,-0.003773,0.007445,0.000022,-0.000008,-0.000002,2.305600e-07
3650,58847.0,1412328,-0.003782,0.007443,0.000023,-0.000008,-0.000002,2.304893e-07
3651,58848.0,1412352,-0.003790,0.007441,0.000023,-0.000008,-0.000002,2.304177e-07


**Conclusion**<br>
Loading daily integration with load_ast_data works.<br>
Integration agrees with JPL to tolerance of **7.9E-7 AU**.

### Splined Asteroid DataFrame

In [17]:
# Load the JPL data run at 3 hour intervals
df_earth_jpl_3h = load_pos_jpl(body_name='earth', dir_name=dir_name_hourly)
df_ast_jpl_3h = load_ast_jpl(ast_num0=1, ast_num1=16, dir_name=dir_name_hourly)

# Load observation from palomar at 3h intervals
df_obs_jpl_3h = load_obs_ast_jpl(ast_num0=1, ast_num1=16, observer_name='palomar', dir_name=dir_name_hourly)

In [18]:
# Inputs for spline_ast_data()
n0 = 1
n1 = 16
mjd = df_earth_jpl_3h.mjd.values

In [19]:
# Spline asteroid data on the same schedule as JPL
df_ast_out, df_earth_out, df_sun_out = spline_ast_data(n0=n0, n1=n1, mjd=mjd)

In [20]:
df_ast_out

,asteroid_num,mjd,time_key,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f
0,1,55197.000,1324728,-1.660334,-2.123236,0.238962,0.007615,-0.007150,-0.001627,2.765732,0.079223,0.184765,1.403138,1.268764,1.371390
1,1,55197.125,1324731,-1.659382,-2.124130,0.238759,0.007618,-0.007146,-0.001627,2.765732,0.079223,0.184765,1.403138,1.268763,1.371879
2,1,55197.250,1324734,-1.658429,-2.125023,0.238556,0.007622,-0.007142,-0.001628,2.765732,0.079223,0.184765,1.403138,1.268761,1.372367
3,1,55197.375,1324737,-1.657476,-2.125915,0.238352,0.007625,-0.007138,-0.001628,2.765731,0.079223,0.184765,1.403138,1.268760,1.372855
4,1,55197.500,1324740,-1.656523,-2.126807,0.238149,0.007628,-0.007134,-0.001628,2.765731,0.079223,0.184765,1.403138,1.268758,1.373344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467467,16,58848.500,1412364,2.522975,-0.473791,-0.045683,0.001437,0.011240,-0.000566,2.922643,0.133480,0.054046,2.618716,-2.286325,-0.520137
467468,16,58848.625,1412367,2.523154,-0.472386,-0.045754,0.001431,0.011242,-0.000566,2.922643,0.133480,0.054046,2.618716,-2.286324,-0.519588
467469,16,58848.750,1412370,2.523332,-0.470981,-0.045825,0.001426,0.011243,-0.000566,2.922643,0.133480,0.054046,2.618716,-2.286323,-0.519038
467470,16,58848.875,1412373,2.523510,-0.469575,-0.045895,0.001420,0.011244,-0.000565,2.922643,0.133480,0.054046,2.618716,-2.286323,-0.518488


In [21]:
# Run comparison on asteroids
compare_df_vec(df_mse=df_ast_out, df_jpl=df_ast_jpl_3h, name='asteroids')

Mean absolute error for df_asteroids_mse vs. df_asteroids_jpl:
mjd: 0.00e+00 days
  q: 7.97e-07 AU     (max 3.52e-06)
  v: 2.60e-09 AU/day (rel 2.43e-07)


In [22]:
df_earth_out

,mjd,time_key,qx,qy,qz,vx,vy,vz,a,e,inc,Omega,omega,f
0,55197.000,1324728,-0.179770,0.970346,-0.000018,-0.017202,-0.003150,8.432308e-07,0.999049,0.015785,0.000050,2.230747,-0.423944,-0.056025
1,55197.125,1324731,-0.181920,0.969950,-0.000018,-0.017195,-0.003188,8.527142e-07,0.999043,0.015778,0.000050,2.226164,-0.420203,-0.052986
2,55197.250,1324734,-0.184069,0.969550,-0.000018,-0.017188,-0.003225,8.615735e-07,0.999038,0.015772,0.000051,2.222007,-0.416898,-0.049936
3,55197.375,1324737,-0.186217,0.969144,-0.000018,-0.017181,-0.003262,8.697954e-07,0.999033,0.015767,0.000051,2.218268,-0.414015,-0.046875
4,55197.500,1324740,-0.188364,0.968734,-0.000018,-0.017174,-0.003300,8.773666e-07,0.999029,0.015763,0.000051,2.214936,-0.411541,-0.043806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29212,58848.500,1412364,-0.161515,0.978013,-0.000019,-0.017273,-0.002832,7.022431e-07,1.000439,0.017185,0.000050,2.734436,-0.919592,-0.082955
29213,58848.625,1412367,-0.163673,0.977657,-0.000018,-0.017267,-0.002870,6.883381e-07,1.000420,0.017166,0.000050,2.749502,-0.934274,-0.081115
29214,58848.750,1412370,-0.165831,0.977296,-0.000018,-0.017261,-0.002908,6.742523e-07,1.000400,0.017147,0.000049,2.765018,-0.949420,-0.079260
29215,58848.875,1412373,-0.167989,0.976930,-0.000018,-0.017254,-0.002946,6.599960e-07,1.000380,0.017128,0.000049,2.780989,-0.965036,-0.077391


In [23]:
# Run comparison on earth
compare_df_vec(df_mse=df_earth_out, df_jpl=df_earth_jpl_3h, name='earth')

Mean absolute error for df_earth_mse vs. df_earth_jpl:
mjd: 0.00e+00 days
  q: 3.92e-06 AU     (max 1.13e-05)
  v: 8.57e-07 AU/day (rel 4.98e-05)


**Conclusion**<br>
Cubic splining of daily integration with spline_ast_data works.<br>
Integration agrees with JPL to tolerance of **3.9E-6 AU**.<br>
The spline has introduced slightly more error, but it is still very small.

### Build Splined Observation & Compare vs. JPL

In [24]:
df_obs_jpl_3h

,asteroid_num,mjd,JulianDate,time_key,RA_jpl,DEC_jpl,ux_jpl,uy_jpl,uz_jpl,RA_apparent,DEC_apparent,delta,delta_dot,light_time
0,1,55197.000,2455197.500,1324728,243.214830,-17.106252,-0.430710,-0.899809,0.069515,243.357955,-17.132205,3.437890,-12.102732,28.592060
1,1,55197.125,2455197.625,1324731,243.266342,-17.117471,-0.429917,-0.900191,0.069477,243.409452,-17.143375,3.437011,-12.282233,28.584749
2,1,55197.250,2455197.750,1324734,243.318051,-17.128756,-0.429121,-0.900574,0.069438,243.461181,-17.154603,3.436114,-12.585595,28.577286
3,1,55197.375,2455197.875,1324737,243.369682,-17.140123,-0.428325,-0.900956,0.069397,243.512878,-17.165916,3.435195,-12.849298,28.569644
4,1,55197.500,2455198.000,1324740,243.420999,-17.151530,-0.427533,-0.901335,0.069354,243.564287,-17.177279,3.434263,-12.933136,28.561893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29212,16,58848.500,2458849.000,1412364,333.918987,-11.709509,0.879482,-0.475698,-0.014963,334.177362,-11.612170,3.052380,16.859446,25.385873
29213,16,58848.625,2458849.125,1412367,333.967430,-11.692635,0.879899,-0.474925,-0.014984,334.225801,-11.595248,3.053587,16.591743,25.395910
29214,16,58848.750,2458849.250,1412370,334.015569,-11.675789,0.880313,-0.474156,-0.015003,334.273964,-11.578362,3.054780,16.504155,25.405834
29215,16,58848.875,2458849.375,1412373,334.063332,-11.658913,0.880724,-0.473391,-0.015020,334.321743,-11.561454,3.055976,16.637661,25.415774


In [25]:
df_obs_mse = spline_ast_obs(df_ast=df_ast_out, df_earth=df_earth_out, site_name='palomar')

In [26]:
df_obs_mse

,asteroid_num,mjd,time_key,ra,dec,ux,uy,uz,delta
0,1,55197.000,1324728,243.214585,-17.106171,-0.430714,-0.899807,0.069516,3.437893
1,1,55197.125,1324731,243.266093,-17.117389,-0.429921,-0.900189,0.069477,3.437014
2,1,55197.250,1324734,243.317799,-17.128673,-0.429125,-0.900572,0.069438,3.436116
3,1,55197.375,1324737,243.369429,-17.140039,-0.428329,-0.900954,0.069397,3.435197
4,1,55197.500,1324740,243.420746,-17.151444,-0.427537,-0.901333,0.069354,3.434265
...,...,...,...,...,...,...,...,...,...
467467,16,58848.500,1412364,333.918680,-11.709608,0.879479,-0.475703,-0.014963,3.052373
467468,16,58848.625,1412367,333.967124,-11.692733,0.879896,-0.474929,-0.014984,3.053580
467469,16,58848.750,1412370,334.015266,-11.675887,0.880311,-0.474160,-0.015003,3.054773
467470,16,58848.875,1412373,334.063030,-11.659011,0.880722,-0.473396,-0.015020,3.055968


In [27]:
compare_df_obs(df_obs_mse, df_obs_jpl_3h, name='asteroid')

Mean absolute error for asteroid observations: MSE vs. JPL
mjd: 0.00e+00 days
Angle Difference: jpl vs. mse
Mean  :   0.000271 deg (   0.974 seconds)
Median:   0.000277 deg (   0.999 seconds)
Max   :   0.000899 deg (   3.236 seconds)


**Conclusion**<br>
df_obs_mse() accurately does an end to end calculation of the RA / DEC of an asteroid.<br>
It is very flexible.  It interpolated the position of the asteroid and the position of the earth,
so it can accept a flexible range of dates to match observations.<br>
It is accurate vs. JPL to a tolerance of **0.97 arc seconds**!